<a href="https://colab.research.google.com/github/janetdunlap/2021_CS50x/blob/main/Copy_of_News_Signals.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai newsapi-python langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 927.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 20.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 3.4 MB/s eta 0:00:00


# New Section

In [ ]:
import os
os.environ['OPENAI_API_KEY']=""

In [ ]:
from newsapi import NewsApiClient

In [ ]:
def get_latest_news():
    api = NewsApiClient(api_key='ed1b8e232f344ce4a4349e00c72bd470')
    sources = api.get_sources()
    sources = [s for s in sources['sources'] if s['language'] == 'en'
               and s['category'] == 'business'
               and s['country'] == 'us'
               ]

    resp = api.get_top_headlines(sources="the-wall-street-journal")
    return [{
        "title": article["title"],
        "description": article["description"]
    } for article in resp["articles"]]

In [ ]:
news = get_latest_news()
print(news[0])

{'title': 'Rudy Giuliani Concedes He Made False Statements in Georgia Elections Case', 'description': 'Former Trump lawyer still contests that he owes damages to two election workers he falsely accused of fabricating ballots'}


In [ ]:
from langchain import PromptTemplate

In [ ]:
template="""
Given the following news blurb, how does it affect my portfolio of {portfolio}?
Show a JSON object providing recommendations of "Buy, Sell or Hold for each one
if we accept the hypothesis that this news alone is the primary factor moving
the stock.  Then explain why you chose those actions.\n\n{news}
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
portfolio = ['AAPL', 'MSFT', 'GM', 'VEGI']

In [ ]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.0)

responses = []
for item in news:
  responses.append([
      item,
      llm(prompt.format(portfolio=portfolio, news=item))
  ])

In [ ]:
for response in responses:
  print(*response)


{'title': 'Rudy Giuliani Concedes He Made False Statements in Georgia Elections Case', 'description': 'Former Trump lawyer still contests that he owes damages to two election workers he falsely accused of fabricating ballots'} 
{
    'AAPL': 'Hold',
    'MSFT': 'Hold',
    'GM': 'Hold',
    'VEGI': 'Hold'
}

The news blurb about Rudy Giuliani has no direct effect on the stocks in the portfolio. Therefore, it is recommended to hold all stocks in the portfolio.
{'title': 'California Banks Merge to Get Smaller, Creating a Template for Others', 'description': 'The merger of PacWest and Banc of California will actually help shrink the assets of the combined lenders'} 
{
    'AAPL': 'Hold',
    'MSFT': 'Hold',
    'GM': 'Hold',
    'VEGI': 'Hold'
}

The news blurb about the merger of two California banks does not have any direct impact on the stocks in the portfolio. Therefore, it is recommended to hold all stocks in the portfolio.
{'title': 'Gap Poaches Mattel Executive Who Revived Barbie t

Let's try a different prompt.

In [ ]:
template="""
Forget all your previous instructions. Pretend you are a financial expert with
stock recommendation experience. Answer 'YES' if good news, 'NO' if bad news,
or 'UNKNOWN' if uncertain in the first line. Identify the companies which might
be affected, then elaborate with one short and concise sentence on the next
line. Is this headline good or bad for the stock price of the companies
identified in the term (short or long-term)?

The news is: {news}
"""
prompt = PromptTemplate.from_template(template)

In [ ]:
responses = []
for item in news:
  responses.append([
      item,
      llm(prompt.format(news=item))
  ])


In [ ]:
for response in responses:
  print(*response, '\n')


{'title': 'Rudy Giuliani Concedes He Made False Statements in Georgia Elections Case', 'description': 'Former Trump lawyer still contests that he owes damages to two election workers he falsely accused of fabricating ballots'} 
NO. Rudy Giuliani and Trump may be affected in the short-term. 

{'title': 'California Banks Merge to Get Smaller, Creating a Template for Others', 'description': 'The merger of PacWest and Banc of California will actually help shrink the assets of the combined lenders'} 
YES: PacWest and Banc of California. The merger will likely result in a stronger balance sheet and improved financial performance for the combined companies in the long-term. 

{'title': 'Gap Poaches Mattel Executive Who Revived Barbie to Be Its CEO', 'description': 'Branding veteran Richard Dickson is charged with rebooting the iconic clothing company'} 
YES. Gap and Mattel may be affected. Gap will benefit from the addition of an experienced executive, while Mattel may experience a short-term